In [1]:
import pystac_client
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
from itertools import compress
import pandas as pd
from shapely import wkt
import geopandas as gpd
import plotly.express as px

# opening STAC catalog

catalog = pystac_client.Client.open(
    #"https://storage.googleapis.com/dgds-data-public/coclico/coclico-stac/catalog.json" # cloud API
    r"p:\1000545-054-globalbeaches\15_GlobalCoastalAtlas\coclicodata\current\catalog.json" # P drive API 
)
#catalog

C:\Users\heijd_de\AppData\Local\Temp\ipykernel_3572\3853560242.py:8: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd


In [3]:
href_dr = catalog.get_child("shore_mon_drivers").assets["data"].href
ds_dr = xr.open_zarr(href_dr)
ds_dr

<xarray.Dataset>
Dimensions:                (stations: 57924)
Coordinates:
    continent              (stations) |S13 dask.array<chunksize=(14481,), meta=np.ndarray>
    country                (stations) |S27 dask.array<chunksize=(14481,), meta=np.ndarray>
    country_id             (stations) |S7 dask.array<chunksize=(28962,), meta=np.ndarray>
    hotspot_id             (stations) |S31 dask.array<chunksize=(14481,), meta=np.ndarray>
    lat                    (stations) float64 dask.array<chunksize=(28962,), meta=np.ndarray>
    lon                    (stations) float64 dask.array<chunksize=(28962,), meta=np.ndarray>
    transect_geom          (stations) |S57 dask.array<chunksize=(7241,), meta=np.ndarray>
    transect_id            (stations) |S15 dask.array<chunksize=(14481,), meta=np.ndarray>
Dimensions without coordinates: stations
Data variables:
    changerate             (stations) float64 dask.array<chunksize=(28962,), meta=np.ndarray>
    date_nourishment       (stations) |S30 dask.array<chunksize=(14481,), meta=np.ndarray>
    ldb_type               (stations) |S14 dask.array<chunksize=(14481,), meta=np.ndarray>
    littoraldb_id_conf     (stations) |S6 dask.array<chunksize=(28962,), meta=np.ndarray>
    nourishment_id_conf    (stations) |S6 dask.array<chunksize=(28962,), meta=np.ndarray>
    reclamation_id_conf    (stations) |S6 dask.array<chunksize=(28962,), meta=np.ndarray>
    seasonal_displacement  (stations) float64 dask.array<chunksize=(28962,), meta=np.ndarray>
    seasonal_id_conf       (stations) |S7 dask.array<chunksize=(28962,), meta=np.ndarray>
    t_max_seasonal_sp      (stations) float64 dask.array<chunksize=(28962,), meta=np.ndarray>
    t_min_seasonal_sp      (stations) float64 dask.array<chunksize=(28962,), meta=np.ndarray>
    t_recl_construction    (stations) float64 dask.array<chunksize=(28962,), meta=np.ndarray>
Attributes: (12/21)
    AUTHOR:              Van der Heijden, D., Luijendijk, A. & Kras, E.
    CITATION:            van der Heijden, D. (2022). Global identification an...
    COMMENT:             
    CRS:                 EPSG:4326
    Conventions:         CF-1.8
    DESCRIPTION:         The Shoreline Monitor Drivers dataset aims to identi...
    ...                  ...
    SPATIAL_EXTENT:      [-180, -90, 180, 90]
    TAGS:                ['civil engineering', 'shoreline dynamics', 'big dat...
    TEMPORAL_EXTENT:     ['1984-01-01T00:00:00Z', '2022-01-01T00:00:00Z']
    TITLE:               The Shoreline Monitor Drivers
    TITLE_ABBREVIATION:  shore_mon_drivers
    UNITS:

In [4]:
country = [b'Netherlands', b'Belgium']
indices = np.where(ds_dr['country'].isin(country))[0]
df_nl = ds_dr.isel(stations = indices).to_dataframe().reset_index()

df_nl['seasonal_id_conf']

0      b'unknown'
1         b'high'
2       b'medium'
3         b'high'
4         b'high'
          ...    
508        b'low'
509     b'medium'
510        b'low'
511       b'high'
512    b'unknown'
Name: seasonal_id_conf, Length: 513, dtype: object

In [5]:


decode_cols = ['seasonal_id_conf', 'littoraldb_id_conf', 'nourishment_id_conf', 'reclamation_id_conf', 'transect_id']
for dc in decode_cols:
    df_nl[dc] = df_nl[dc].str.decode("utf-8")


# Add a fake data point with the gray category and null location
fake_data = pd.DataFrame({'littoraldb_id_conf': ['medium'], 'lat': [None], 'lon': [None]})
df_nl = pd.concat([df_nl, fake_data])

label_colors = {'low': 'red', 'medium':'orange', 'high': 'green'}
order = list(label_colors.keys())

fig = px.scatter_mapbox(df_nl, 
                    lat='lat', 
                    lon = 'lon',
                    color='littoraldb_id_conf',
                    color_discrete_map= label_colors,
                    hover_data= ['transect_id'],
                    category_orders= {'littoraldb_id_conf': order},
                    center={'lat': 52.5, 'lon': 5},
                    mapbox_style='carto-positron', 
                    zoom=6.5,
                    width = 800,
                    height = 1000)

fig.update_layout(legend=dict(
    title="Confidence Level",
    ),
    title = 'Littoral Drift Barrier Identification')


fig.show()

In [6]:
label_colors = {'low': 'red', 'medium':'orange', 'high': 'green'}

fig = px.scatter_mapbox(df_nl, 
                    lat='lat', 
                    lon = 'lon',
                    color='nourishment_id_conf',
                    color_discrete_map= label_colors,
                    category_orders= {'nourishment_id_conf': list(label_colors.keys())},
                    hover_data= ['transect_id'],
                    center={'lat': 52.5, 'lon': 5},
                    mapbox_style='carto-positron', 
                    zoom=6.5,
                    width = 800,
                    height = 1000)

fig.update_layout(legend=dict(
    title="Confidence Level",
    ),
    title = 'Nourishment Identification')


fig.show()

In [8]:
label_colors = {'low': 'red', 'medium':'orange', 'high': 'green', 'unknown': 'grey'}

fig = px.scatter_mapbox(df_nl, 
                    lat='lat', 
                    lon = 'lon',
                    color='seasonal_id_conf',
                    color_discrete_map= label_colors,
                    category_orders= {'seasonal_id_conf': list(label_colors.keys())},
                    hover_data= ['transect_id'],
                    center={'lat': 52.5, 'lon': 5},
                    mapbox_style='carto-positron', 
                    zoom=6.5,
                    width = 800,
                    height = 1000)

fig.update_layout(legend=dict(
    title="Confidence Level",
    ),
    title = 'Seasonality Identification')

fig.show()

In [9]:
country = [b'Qatar', b'United Arab Emirates']
indices = np.where(ds_dr['country'].isin(country))[0]
df_q = ds_dr.isel(stations = indices).to_dataframe().reset_index()

for dc in decode_cols:
    df_q[dc] = df_q[dc].str.decode("utf-8")

fig = px.scatter_mapbox(df_q, 
                    lat='lat', 
                    lon = 'lon',
                    color='reclamation_id_conf',
                    color_discrete_map= label_colors,
                    category_orders= {'reclamation_id_conf': list(label_colors.keys())},
                    hover_data= ['transect_id'],
                    center={'lat': df_q['lat'].mean(), 'lon': df_q['lon'].mean()},
                    mapbox_style='carto-positron', 
                    zoom=5.5,
                    width = 800,
                    height = 1000)


fig.update_layout(legend=dict(
    title="Confidence Level",
    ),
    title = 'Reclamation Identification')

fig.show()